# Inference Notebook Template

**What this does:**
1. **Helper** (`save_results`): dumps any single `dict` into a timestamped JSON file under `results/`.
2. **Prompt & image**: only one each—just swap in your own strings/paths.
3. **Model loading**: picks the chosen variant from HF.
4. **Inference**: calls `.generate()` on one image + prompt.
5. **Output**: prints the output and writes all metadata + result into JSON.

In [5]:
!pip install git+https://github.com/huggingface/transformers accelerate
!pip install qwen-vl-utils[decord]==0.0.8
!pip install -qqq num2words

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-67w9ccth
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-67w9ccth
  Resolved https://github.com/huggingface/transformers to commit fa3c3f9cab1c45b449bd57e238c511c79637e314
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s e

## 📚 Helper: Save any results dict to JSON

In [6]:
import json, os
from datetime import datetime

def save_results(data: dict,
                 model_name: str,
                 variant: str,
                 output_dir: str = "results"):
    # Ensure nested directories are created
    model_dir = os.path.join(output_dir, model_name)
    os.makedirs(model_dir, exist_ok=True)

    ts       = datetime.now().strftime("%Y%m%d_%H%M%S")
    fname    = f"{variant}_{ts}.json"
    out_path = os.path.join(model_dir, fname)

    with open(out_path, "w") as f:
        json.dump(data, f, indent=4)
    print(f"✅ Saved results to {out_path}")


# Variant: Qwen2.5-VL-3B-Instruct
https://huggingface.co/Qwen/Qwen2.5-VL-3B-Instruct

## 1️⃣ Prompt

In [7]:
prompt = "What is this best described as? Choose ONE from: 'Art', 'Graffiti', 'Vandalism', 'Activism', 'Advertisement', 'Other'."

## 2️⃣ Load Processor and Model

In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from glob import glob
from PIL import Image
import torch
import os

MODEL_NAME = "Qwen/Qwen2.5-VL"
VARIANT    = "3B-Instruct"
repo_id    = f"{MODEL_NAME}-{VARIANT}"

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    repo_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(repo_id)

input_dir = "/content/input"
image_extensions = ["*.png", "*.jpg", "*.jpeg", "*.webp", "*.bmp"]

image_paths = []
for ext in image_extensions:
    image_paths.extend(glob(os.path.join(input_dir, ext)))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/5.70k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

## 3️⃣ Inference

In [13]:
def infer_img(image_path: str, prompt: str):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    messages = [{
        "role": "user",
        "content": [
            {"type": "image", "image": f"file://{image_path}"},
            {"type": "text", "text": prompt}
        ]
    }]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to("cuda")

    with torch.inference_mode():
        generated_ids = model.generate(**inputs, max_new_tokens=512, temperature=0)
        trimmed = [o[len(i):] for i, o in zip(inputs.input_ids, generated_ids)]
        output = processor.batch_decode(trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    return output.strip()


## 4️⃣ Package & Save to JSON

In [14]:
results = []

for image_path in image_paths:
    try:
        output = infer_img(image_path, prompt)
        print(f"{image_path} → {output}")

        result = {
            "image_path": image_path,
            "model":      MODEL_NAME,
            "variant":    VARIANT,
            "prompt":     prompt,
            "output":     output
        }

        results.append(result)

    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")

# Save after all
save_results(results, MODEL_NAME.split("/")[-1], VARIANT)


✅ Saved results to results/Qwen2.5-VL/3B-Instruct_20250502_112807.json


In [15]:
!sudo rm -rf ./results/*


# Variant: Qwen2.5-VL-7B-Instruct

https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct

In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from glob import glob
from PIL import Image
import torch
import os

MODEL_NAME = "Qwen/Qwen2.5-VL"
VARIANT    = "7B-Instruct"
repo_id    = f"{MODEL_NAME}-{VARIANT}"

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    repo_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(repo_id)

input_dir = "/content/input"
image_extensions = ["*.png", "*.jpg", "*.jpeg", "*.webp", "*.bmp"]

image_paths = []
for ext in image_extensions:
    image_paths.extend(glob(os.path.join(input_dir, ext)))


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/57.6k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

## 3️⃣ Inference

In [ ]:
def infer_img(image_path: str, prompt: str):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    messages = [{
        "role": "user",
        "content": [
            {"type": "image", "image": f"file://{image_path}"},
            {"type": "text", "text": prompt}
        ]
    }]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to("cuda")

    with torch.inference_mode():
        generated_ids = model.generate(**inputs, max_new_tokens=512, temperature=0)
        trimmed = [o[len(i):] for i, o in zip(inputs.input_ids, generated_ids)]
        output = processor.batch_decode(trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    return output.strip()


## 4️⃣ Package & Save to JSON

In [ ]:
results = []

for image_path in image_paths:
    try:
        output = infer_img(image_path, prompt)
        print(f"{image_path} → {output}")

        result = {
            "image_path": image_path,
            "model":      MODEL_NAME,
            "variant":    VARIANT,
            "prompt":     prompt,
            "output":     output
        }

        results.append(result)

    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")

# Save after all
save_results(results, MODEL_NAME.split("/")[-1], VARIANT)


✅ Saved results to results/Qwen2-VL/7B-Instruct_20250502_110816.json
